# Создание агента

:::note

В разделе затрагиваются слдующие основные понятия:

- [Чат-модели](/docs/concepts/#chat-models)
- [Инструменты](/docs/concepts/#tools)
- [Агенты](/docs/concepts/#agents)

:::

Языковые модели просто генерируют текст и не могут выполнять действия.
Разработка *агентов* — это один из основных сценариев использования GigaChain.

Агенты — системы, использующие LLM для рассуждений и определения действий, которые нужно предпринять, а также входных данных, которые нужно при этом использовать.

Результаты этих действий затем могут быть переданы обратно агенту, чтобы он определил, нужно ли делать что-то еще или можно закончить работу.

В разделе приведен пример агента, который может взаимодействовать с несколькими инструментами: локальной базой данных и поисковой системой.
С агентом можно вести разговор и наблюдать как он вызывает инструменты.

## Пример агента

Пример ниже содержит код работающего агента, который использует модель для определения того, какой инструмент нужно вызвать.
Агент подготовлен для работы с поисковым инструментом и обладает разговорной памятью, что позволяет использвоать его в роли развитого чат-бота.

Ниже в разделе приводится пошаговый разбор каждого из компонтов агента.

In [76]:
# Импортирование необходимой функциональности
from langchain_community.chat_models.gigachat import GigaChat
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import create_react_agent

# Создание агента
memory = SqliteSaver.from_conn_string(":memory:")
model = GigaChat(
    credentials="<авторизационные_данные>",
    scope="GIGACHAT_API_PERS",
    model="GigaChat-Pro",
    verify_ssl_certs=False,
)
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Использование агента
config = {"configurable": {"thread_id": "abc100"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Привет! Меня зову Вася. Я живу в Москве")]},
    config,
):
    print(chunk)
    print("----")

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Покажи данные о погоде в моем городе")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Здравствуйте! Очень приятно познакомиться, Вася из Москвы. Чем я могу вам помочь сегодня?', additional_kwargs={'functions_state_id': 'c94ca48e-f423-4bdc-8073-156a4e48e1e3'}, response_metadata={'token_usage': Usage(prompt_tokens=102, completion_tokens=27, total_tokens=129), 'model_name': 'GigaChat-Pro:1.0.26.8', 'finish_reason': 'stop'}, id='run-4a31e45c-5274-432a-86e9-a31ff854c68c-0')]}}
----


{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': {'query': 'погода в Москве'}}, 'functions_state_id': 'f1a100da-7085-4fb0-b54d-f36b988f0713'}, response_metadata={'token_usage': Usage(prompt_tokens=157, completion_tokens=35, total_tokens=192), 'model_name': 'GigaChat-Pro:1.0.26.8', 'finish_reason': 'function_call'}, id='run-b1ef0f7b-ee72-4097-b9e9-c2b8b3ae6574-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'погода в Москве'}, 'id': '3a2fc4ff-938d-4eb4-a4c6-fa00781b5afe'}])]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://pogoda.mail.ru/prognoz/moskva/14dney/", "content": "\\u0423\\u0437\\u043d\\u0430\\u0439\\u0442\\u0435, \\u043a\\u0430\\u043a\\u0430\\u044f \\u043f\\u043e\\u0433\\u043e\\u0434\\u0430 \\u043e\\u0436\\u0438\\u0434\\u0430\\u0435\\u0442\\u0441\\u044f \\u0432 \\u041c\\u043e\\u0441\\u043a\\u0432\\u0435 \\u043d\\u0430 \\u0431\\u043b\\u0438\\u0436\

## Подготовка к разработке

### Jupyter-блокноты

Это руководство, как и большинство других в документации, использует [Jupyter-блокноты](https://jupyter.org/). Они отлично подходят для изучения работы с LLM-системами, так как предоставляют интерактивную среду для работы с руководствами и позволяют работать с непредвиденными ситуациями: недоступностью API, нетипичным выводом и другими.

Подробнее об установке jupyter -- в [официальной документации](https://jupyter.org/install).

### Установка

Для установки GigaChain выполните команды:

In [ ]:
pip install -U gigachain-community gigagraph tavily-python

Подробнее об установке — в разделе [Установка](https://developers.sber.ru/docs/ru/gigachain/get-started/installation).

<!--
### LangSmith

Многие приложения, которые вы создаете с помощью LangChain, будут содержать несколько шагов с многократными вызовами LLM.
По мере усложнения этих приложений становится важно иметь возможность инспектировать, что именно происходит внутри вашей цепочки или агента.
Лучший способ сделать это — с помощью [LangSmith](https://smith.langchain.com).

После регистрации по ссылке выше, убедитесь, что вы установили переменные среды для начала ведения журнала трассировок:

```shell
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_API_KEY="..."
```

Или, если вы работаете в блокноте, вы можете установить их с помощью:

```python
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
```
-->

### Tavily

В качестве инструмента в примере используется поисковый движок [Tavily](/docs/integrations/tools/tavily_search).
Для работы с ним вам нужно получить и задать ключ API:

```bash
export TAVILY_API_KEY="..."
```

В Jupyter-блокноте его можно использовать так:

In [ ]:
# Используйте эту ячейку, чтобы задать ключ в блокноте
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

## Определение инструментов

Сначала создайте инструменты, которые будет использовать агент.
Основным инструментом в примере выступает поисковый движок [Tavily](/docs/integrations/tools/tavily_search).
GigaChain предоставляет интеграцию для простого использования поисковой системы Tavily в качестве инструмента.

In [31]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("Узнай на weatherapi погоду в Москве")
print(search_results)
# Если нужно, вы можете создать другие инструменты.
# После создания всех необходимых инструментов
# их можно сохранить в списке, к которому можно обращаться позднее.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Moscow', 'region': 'Moscow City', 'country': 'Russia', 'lat': 55.75, 'lon': 37.62, 'tz_id': 'Europe/Moscow', 'localtime_epoch': 1722510905, 'localtime': '2024-08-01 14:15'}, 'current': {'last_updated_epoch': 1722510900, 'last_updated': '2024-08-01 14:15', 'temp_c': 19.9, 'temp_f': 67.7, 'is_day': 1, 'condition': {'text': 'Patchy light drizzle', 'icon': '//cdn.weatherapi.com/weather/64x64/day/263.png', 'code': 1150}, 'wind_mph': 11.4, 'wind_kph': 18.4, 'wind_degree': 284, 'wind_dir': 'WNW', 'pressure_mb': 1001.0, 'pressure_in': 29.56, 'precip_mm': 0.51, 'precip_in': 0.02, 'humidity': 59, 'cloud': 70, 'feelslike_c': 19.9, 'feelslike_f': 67.7, 'windchill_c': 19.9, 'windchill_f': 67.7, 'heatindex_c': 19.9, 'heatindex_f': 67.7, 'dewpoint_c': 11.8, 'dewpoint_f': 53.2, 'vis_km': 5.0, 'vis_miles': 3.0, 'uv': 4.0, 'gust_mph': 13.3, 'gust_kph': 21.3}}"}, {'url': 'https://www.weatherapi.com/weather/q/moscow-2145091', 'cont

## Использование языковых моделей

Пример ниже показвыает как исползовать языковую модель для вызова инструментов.
Кроме моделей GigaChat, GigaChain позволяет использовать и другие модели.

In [9]:
# | output: false
# | echo: false

from langchain_community.chat_models.gigachat import GigaChat

model = GigaChat(
    credentials="<авторизационные_данные>",
    scope="GIGACHAT_API_PERS",
    model="GigaChat-Pro",
    verify_ssl_certs=False,
)

Для вызова модели передайте ей список сообщений.
По умолчанию ответом будет строка `content`.

In [10]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="Привет!")])
response.content

'Здравствуйте! Готова ответить на ваши вопросы.'

Используйте метод `.bind_tools`, чтобы дать модели знать к каким инструментам она может обращаться.

In [11]:
model_with_tools = model.bind_tools(tools)

Сначала, посмотрите как ответит модель на обчное сообщение.
Ответ модели можно найти в двух полях `content` и `tool_calls`.

In [12]:
response = model_with_tools.invoke([HumanMessage(content="Привет!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Здравствуйте! Чем я могу вам помочь?
ToolCalls: []


Теперь попробуйте вызвать модель с помощью запроса, который предполагает использование инструмента.

In [35]:
response = model_with_tools.invoke(
    [HumanMessage(content="Какая погода в Москве, спроси у weatherapi?")]
)

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'Какая погода в Москве?'}, 'id': '38c51d05-adf3-42d7-85b0-8af53818e03e'}]


Поле ContentString теперь пустое, но по содержимому массива ToolCalls видно, что модель хочет, чтобы вы вызвали инструмент поиска Tavily.

Модель не может самостоятельно вызвать инструмент, для этого нужно создать агента.

## Создание агента

Для создания агента используйте [GigaGraph](/docs/concepts/#langgraph) — инструмент, который предоставляет высокоуровневый интерфейс для создания агентов, а также дает доступ к низкоуровневым инструментам, которые дают полный контроль над логикой работы агента.

Инициализируйте агент с моделью и набором инструментов.

:::note

В агент можно передавать `model`, а не `model_with_tools`, т.к. метод `create_react_agent` самостоятельно вызывает `.bind_tools`.

:::

In [38]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

## Запуск агента

Теперь вы можете проверить работу агента на нескольких запросах.

Сейчас агент не сохраняет информацию о состоянии и не помнит историю взаимодействия с пользователем.

В результате работы агент возвращает итоговое состояние, которое кроме выходных данных также содержит все входные данные.
Как получить только выходные данные показано в примерах ниже.

Посмотрите как агент отвечает, когда не нужно вызывать инструменты:

In [39]:
response = agent_executor.invoke({"messages": [HumanMessage(content="Привет!")]})

response["messages"]

[HumanMessage(content='Привет!', id='e8329188-248d-4c3f-8b5c-7fa71b699dbf'),
 AIMessage(content='Здравствуйте! Чем я могу вам помочь?', additional_kwargs={'functions_state_id': '5ea42efe-df02-4b67-9b42-f8c7a7a513c8'}, response_metadata={'token_usage': Usage(prompt_tokens=91, completion_tokens=17, total_tokens=108), 'model_name': 'GigaChat-Pro:1.0.26.8', 'finish_reason': 'stop'}, id='run-1778eb8f-3ed8-40e5-becb-f7c6cbebc323-0')]

<!--
In order to see exactly what is happening under the hood (and to make sure it's not calling a tool) we can take a look at the [LangSmith trace](https://smith.langchain.com/public/28311faa-e135-4d6a-ab6b-caecf6482aaa/r)
-->

Попробуйте обратиться к агенту с запросом, который предполагает вызов инструмента:

In [44]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="Какая погода в Москве, узнай на weatherapi?")]}
)
response["messages"]

[HumanMessage(content='Какая погода в Москве, узнай на weatherapi?', id='2f10751b-3b3e-43fa-9872-9153e006551d'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': {'query': 'Какая погода в Москве, узнай на weatherapi?'}}, 'functions_state_id': 'e6f2cbba-7964-433f-8f1e-972c3badd13d'}, response_metadata={'token_usage': Usage(prompt_tokens=101, completion_tokens=44, total_tokens=145), 'model_name': 'GigaChat-Pro:1.0.26.8', 'finish_reason': 'function_call'}, id='run-ef497eef-d26c-41ae-853f-33dc6ad30657-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Какая погода в Москве, узнай на weatherapi?'}, 'id': 'b3f7704b-a8fb-4ff9-8a00-3111a14b1ad8'}]),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'\\u041c\\u043e\\u0441\\u043a\\u0432\\u0430\', \'region\': \'Moscow City\', \'country\': \'\\u0420\\u043e\\u0441\\u0441\\u0438\\u044f\', \'lat\': 55.75, \'lon\': 37

<!--
We can check out the [LangSmith trace](https://smith.langchain.com/public/f520839d-cd4d-4495-8764-e32b548e235d/r) to make sure it's calling the search tool effectively.
-->

## Потоковая передача сообщений

В предыдущих примерах агент вызывается с помощью метода `.invoke` и возвращает итоговой ответ.
Это может занимать продолжительное время, если агент в процессе работы выполняет множество шагов.
Для демонстрации промежуточных результатов вы можете передавать сообщения по мере их возникновения.

In [45]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Какая погода в Москве, узнай на weatherapi?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': {'query': 'Какая погода в Москве, узнай на weatherapi?'}}, 'functions_state_id': '1c01b637-c2c1-4e2e-8683-821b103a779d'}, response_metadata={'token_usage': Usage(prompt_tokens=101, completion_tokens=44, total_tokens=145), 'model_name': 'GigaChat-Pro:1.0.26.8', 'finish_reason': 'function_call'}, id='run-ffb8a6a0-c0cb-4860-b3da-897a6059c72d-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Какая погода в Москве, узнай на weatherapi?'}, 'id': '58ea597a-c72e-43cd-bad1-1677b5e3d435'}])]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'\\u041c\\u043e\\u0441\\u043a\\u0432\\u0430\', \'region\': \'Moscow City\', \'country\': \'\\u0420\\u043e\\u0441\\u0441\\u0438\\u044f\', \'lat\': 55.75, \'lon\': 37.62, \'tz_id\': \'Europe/Moscow\', \'localtime_epoch\': 1722

## Потоковая передача токенов

Кроме сообщений вы также можете использовать потоковую передачу токенов.
Для этого используйте метод `.astream_events`.

:::important

Метод `.astream_events` работает в версиях Python 3.11 и выше.

:::

In [46]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="Какая погода в Москве, узнай на weatherapi?")]},
    version="v1",
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Назначается при создании агента с помощью метода `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Назначается при создании агента с помощью метода `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

--
Starting tool: tavily_search_results_json with inputs: {'query': 'Какая погода в Москве, узнай на weatherapi?'}
Done tool: tavily_search_results_json
Tool output was: [{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Moscow', 'region': 'Moscow City', 'country': 'Russia', 'lat': 55.75, 'lon': 37.62, 'tz_id': 'Europe/Moscow', 'localtime_epoch': 1722511212, 'localtime': '2024-08-01 14:20'}, 'current': {'last_updated_epoch': 1722510900, 'last_updated': '2024-08-01 14:15', 'temp_c': 19.9, 'temp_f': 67.7, 'is_day': 1, 'condition': {'text': 'Patchy light drizzle', 'icon': '//cdn.weatherapi.com/weather/64x64/day/263.png', 'code': 1150}, 'wind_mph': 11.4, 'wind_kph': 18.4, 'wind_degree': 284, 'wind_dir': 'WNW', 'pressure_mb': 1001.0, 'pressure_in': 29.56, 'precip_mm': 0.51, 'precip_in': 0.02, 'humidity': 59, 'cloud': 70, 'feelslike_c': 19.9, 'feelslike_f': 67.7, 'windchill_c': 19.9, 'windchill_f': 67.7, 'heatindex_c': 19.9, 'heatindex_f': 67.7, 'dewpoint_c': 11.8, 'd

## Добавление памяти

Для добавления памяти нужно при вызове агента передать ему чекпойнтер и поле `thread_id`.
С помощью этог поля агент сможет понять к какому разговору нужно вернуться.

In [22]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [23]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [24]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Привет! Меня зовут Вася")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Здравствуйте, Вася! Рада познакомиться. Чем я могу вам помочь сегодня?', additional_kwargs={'functions_state_id': '48672f4f-dc4a-4370-be7e-ffce6d24c883'}, response_metadata={'token_usage': Usage(prompt_tokens=97, completion_tokens=25, total_tokens=122), 'model_name': 'GigaChat-Pro:1.0.26.8', 'finish_reason': 'stop'}, id='run-056a1f85-08a2-417f-b2f6-7117d369b30b-0')]}}
----


In [25]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Как меня зовут?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Вас зовут Вася.', additional_kwargs={'functions_state_id': 'ae930f9b-0a69-46a9-9338-d65ff768f71e'}, response_metadata={'token_usage': Usage(prompt_tokens=148, completion_tokens=12, total_tokens=160), 'model_name': 'GigaChat-Pro:1.0.26.8', 'finish_reason': 'stop'}, id='run-be6d19fb-b89d-4e34-b5a7-6d82ed90b2e2-0')]}}
----


<!--
Example [LangSmith trace](https://smith.langchain.com/public/fa73960b-0f7d-4910-b73d-757a12f33b2b/r)
-->

Для начала нового разговора вам достаточно изменить значение `thread_id`.

In [26]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Как меня зовут?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Извините, но я не имею возможности предоставить вам эту информацию. Моя система не хранит и не обрабатывает персональные данные пользователей.', additional_kwargs={'functions_state_id': '3ce4de5a-95b5-4f4a-b18e-40def1558c82'}, response_metadata={'token_usage': Usage(prompt_tokens=94, completion_tokens=35, total_tokens=129), 'model_name': 'GigaChat-Pro:1.0.26.8', 'finish_reason': 'stop'}, id='run-6a2131a0-1534-4292-b53b-dc40eb68f52d-0')]}}
----


## Смотрите также

Более подробную информацию о разработке агентов ищите в документации [GigaGraph](/docs/concepts/#langgraph).